# HISTORY

* Spirochaeta is a genus of bacteria classified within the phylum Spirochaetes. Included in this dataset are 366 darkfield microscopy images and manually annotated masks which can be used for classification and segmentation purposes. Detecting bacteria in blood could have a huge significance for research in both the medical and computer science field.

#### What is special about this dataset?
* It was gathered and annotated by students (hand-on experience)
* It has more than just one targeted class (blood cell and bacteria were annotated)
* It is highly imbalanced, so naive loss functions would work less properly

# PACKAGES AND LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import time
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
import imageio
from IPython.display import Image
import matplotlib.image as mpimg
from skimage.transform import resize
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import zipfile
from io import BytesIO
from nibabel import FileHolder
from nibabel.analyze import AnalyzeImage
import PIL
from IPython import display
from skimage.morphology import convex_hull_image, erosion
from skimage.morphology import square
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from skimage import data, io, filters
import skimage
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose, LeakyReLU, ReLU
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# PATH PROCESS

#### MAIN PATH

In [ ]:
Bacteria_Path = Path("../input/bacteria-detection-with-darkfield-microscopy/images")
Mask_Path = Path("../input/bacteria-detection-with-darkfield-microscopy/masks")

#### PNG PATH

In [ ]:
Bacteria_PNG = list(Bacteria_Path.glob(r"*.png"))
Mask_PNG = list(Mask_Path.glob(r"*.png"))

In [ ]:
print(Bacteria_PNG[0:5])

In [ ]:
print(Mask_PNG[0:5])

#### TO SERIES

In [ ]:
Bacteria_Series = pd.Series(Bacteria_PNG,name="BACTERIA").astype(str)
Mask_Series = pd.Series(Mask_PNG,name="MASK").astype(str)

In [ ]:
print(Bacteria_Series.head(-1))

In [ ]:
print(Mask_Series.head(-1))

#### TO DATAFRAME

In [ ]:
Main_Set = pd.concat([Bacteria_Series,Mask_Series],axis=1)

In [ ]:
print(Main_Set.head(-1))

# VISION & ANALYSIS

#### FUNCTION

In [ ]:
def simple_reading(image):
    
    Reading_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2RGB)
    
    return Reading_Image

In [ ]:
def gray_reading(image):
    
    Gray_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2GRAY)
    
    return Gray_Image

In [ ]:
def threshold_reading(image):
    
    _,Threshold_Image = cv2.threshold(image,200,255,cv2.THRESH_BINARY)
    
    return Threshold_Image

In [ ]:
def adaptive_threshold_reading(image):
    
    Reading_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2GRAY)
    Adaptive_Image = cv2.adaptiveThreshold(Reading_Image,20,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,9,2)
    
    return Adaptive_Image

In [ ]:
def canny_reading(image):
    
    Canny_Image = cv2.Canny(image,10,100)
    
    return Canny_Image

In [ ]:
def hessian_reading(image,sigma,order):
    
    Reading_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2GRAY)
    Hessian_Image = hessian_matrix(Reading_Image,sigma=sigma,order=order)
    
    Max_IMG,Min_IMG = hessian_matrix_eigvals(Hessian_Image)
    
    return Max_IMG,Min_IMG

In [ ]:
def skeleton_reading(image):
    
    Reading_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2GRAY)
    _,Threshold_Image = cv2.threshold(Reading_Image,200,255,cv2.THRESH_BINARY)
    Array_Image = np.array(Reading_Image > Threshold_Image).astype(int)
    Skeleton_Image = skimage.morphology.skeletonize(Array_Image)
    
    return Skeleton_Image  

In [ ]:
def layers_reading(image,mask):
    
    Reading_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2RGB)
    Reading_Mask = cv2.cvtColor(cv2.imread(mask),cv2.COLOR_BGR2GRAY)
    
    Copy_Main = Reading_Image.copy()
    Copy_Main[Reading_Mask == 1] = [255,0,0]
    Copy_Main[Reading_Mask == 2] = [0,0,255]
    
    Copy_Main_Two = Reading_Image.copy()
    Blend_Image = cv2.addWeighted(Copy_Main,0.5,Copy_Main_Two,0.5,0,Copy_Main_Two)
    
    return Blend_Image

In [ ]:
def drawing_contours_vision(image):
    
    figure,axis = plt.subplots(nrows=1,ncols=2,figsize=(12,12))
    
    Reading_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2RGB)
    Canny_Image = cv2.Canny(Reading_Image,10,100)
    contours,_ = cv2.findContours(Canny_Image,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    Draw_Contours = cv2.drawContours(Reading_Image,contours,-1,(255,0,0),1)
    
    axis[0].imshow(Canny_Image)
    axis[1].imshow(Draw_Contours)

In [ ]:
def simple_reading_vision(image):
    
    figure = plt.figure(figsize=(8,8))
    
    Reading_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2RGB)
    
    plt.imshow(Reading_Image)
    plt.xlabel(Reading_Image.shape)
    plt.ylabel(Reading_Image.size)

In [ ]:
def simple_spectrum_vision(image,cmap_type):
    
    figure = plt.figure(figsize=(8,8))
    
    Reading_Image = cv2.cvtColor(cv2.imread(image),cv2.COLOR_BGR2RGB)
    Reading_Image = Reading_Image[:,:,0]
    
    plt.imshow(Reading_Image,cmap=cmap_type)
    plt.xlabel(Reading_Image.shape)
    plt.ylabel(Reading_Image.size)
    

In [ ]:
def simple_vision(image):
    
    figure = plt.figure(figsize=(8,8))
    
    plt.imshow(image)
    plt.xlabel(image.shape)
    plt.ylabel(image.size)

#### SIMPLE VISION

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

axis[0].set_xlabel(simple_reading(Main_Set["BACTERIA"][10]).shape)
axis[0].set_ylabel(simple_reading(Main_Set["BACTERIA"][10]).size)
axis[0].set_title("BACTERIA")
axis[0].imshow(simple_reading(Main_Set["BACTERIA"][10]))

axis[1].set_xlabel(gray_reading(Main_Set["MASK"][10]).shape)
axis[1].set_ylabel(gray_reading(Main_Set["MASK"][10]).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Main_Set["MASK"][10]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

axis[0].set_xlabel(simple_reading(Main_Set["BACTERIA"][100]).shape)
axis[0].set_ylabel(simple_reading(Main_Set["BACTERIA"][100]).size)
axis[0].set_title("BACTERIA")
axis[0].imshow(simple_reading(Main_Set["BACTERIA"][100]))

axis[1].set_xlabel(gray_reading(Main_Set["MASK"][100]).shape)
axis[1].set_ylabel(gray_reading(Main_Set["MASK"][100]).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Main_Set["MASK"][100]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

axis[0].set_xlabel(simple_reading(Main_Set["BACTERIA"][230]).shape)
axis[0].set_ylabel(simple_reading(Main_Set["BACTERIA"][230]).size)
axis[0].set_title("BACTERIA")
axis[0].imshow(simple_reading(Main_Set["BACTERIA"][230]))

axis[1].set_xlabel(gray_reading(Main_Set["MASK"][230]).shape)
axis[1].set_ylabel(gray_reading(Main_Set["MASK"][230]).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Main_Set["MASK"][230]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

axis[0].set_xlabel(simple_reading(Main_Set["BACTERIA"][311]).shape)
axis[0].set_ylabel(simple_reading(Main_Set["BACTERIA"][311]).size)
axis[0].set_title("BACTERIA")
axis[0].imshow(simple_reading(Main_Set["BACTERIA"][311]))

axis[1].set_xlabel(gray_reading(Main_Set["MASK"][311]).shape)
axis[1].set_ylabel(gray_reading(Main_Set["MASK"][311]).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Main_Set["MASK"][311]))

In [ ]:
figure,axis = plt.subplots(5,5,figsize=(10,10))

for indexing,operations in enumerate(axis.flat):
    
    Picking_Image = simple_reading(Main_Set["BACTERIA"][indexing])
    
    operations.set_xlabel(Picking_Image.shape)
    operations.set_ylabel(Picking_Image.size)
    operations.set_title("BACTERIA")
    operations.imshow(Picking_Image)
    
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(5,5,figsize=(10,10))

for indexing,operations in enumerate(axis.flat):
    
    Picking_Mask = gray_reading(Main_Set["MASK"][indexing])
    
    operations.set_xlabel(Picking_Mask.shape)
    operations.set_ylabel(Picking_Mask.size)
    operations.set_title("MASK")
    operations.imshow(Picking_Mask)
    
plt.tight_layout()
plt.show()

#### THRESHOLD VISION

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(11,11))


Image_Picking = Main_Set["BACTERIA"][15]
Mask_Picking = Main_Set["MASK"][15]

axis[0].set_xlabel(threshold_reading(simple_reading(Image_Picking)).shape)
axis[0].set_ylabel(threshold_reading(simple_reading(Image_Picking)).size)
axis[0].set_title("THRESHOLD")
axis[0].imshow(threshold_reading(simple_reading(Image_Picking)))

axis[1].set_xlabel(gray_reading(Mask_Picking).shape)
axis[1].set_ylabel(gray_reading(Mask_Picking).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Mask_Picking))

axis[2].set_xlabel(simple_reading(Image_Picking).shape)
axis[2].set_ylabel(simple_reading(Image_Picking).size)
axis[2].set_title("BACTERIA")
axis[2].imshow(simple_reading(Image_Picking))

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(11,11))


Image_Picking = Main_Set["BACTERIA"][155]
Mask_Picking = Main_Set["MASK"][155]

axis[0].set_xlabel(threshold_reading(simple_reading(Image_Picking)).shape)
axis[0].set_ylabel(threshold_reading(simple_reading(Image_Picking)).size)
axis[0].set_title("THRESHOLD")
axis[0].imshow(threshold_reading(simple_reading(Image_Picking)))

axis[1].set_xlabel(gray_reading(Mask_Picking).shape)
axis[1].set_ylabel(gray_reading(Mask_Picking).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Mask_Picking))

axis[2].set_xlabel(simple_reading(Image_Picking).shape)
axis[2].set_ylabel(simple_reading(Image_Picking).size)
axis[2].set_title("BACTERIA")
axis[2].imshow(simple_reading(Image_Picking))

#### ADAPTIVE THRESHOLD VISION

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(11,11))


Image_Picking = Main_Set["BACTERIA"][1]
Mask_Picking = Main_Set["MASK"][1]

axis[0].set_xlabel(adaptive_threshold_reading(Image_Picking).shape)
axis[0].set_ylabel(adaptive_threshold_reading(Image_Picking).size)
axis[0].set_title("ADAPTIVE THRESHOLD")
axis[0].imshow(adaptive_threshold_reading(Image_Picking))

axis[1].set_xlabel(gray_reading(Mask_Picking).shape)
axis[1].set_ylabel(gray_reading(Mask_Picking).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Mask_Picking))

axis[2].set_xlabel(simple_reading(Image_Picking).shape)
axis[2].set_ylabel(simple_reading(Image_Picking).size)
axis[2].set_title("BACTERIA")
axis[2].imshow(simple_reading(Image_Picking))

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(11,11))


Image_Picking = Main_Set["BACTERIA"][244]
Mask_Picking = Main_Set["MASK"][244]

axis[0].set_xlabel(adaptive_threshold_reading(Image_Picking).shape)
axis[0].set_ylabel(adaptive_threshold_reading(Image_Picking).size)
axis[0].set_title("ADAPTIVE THRESHOLD")
axis[0].imshow(adaptive_threshold_reading(Image_Picking))

axis[1].set_xlabel(gray_reading(Mask_Picking).shape)
axis[1].set_ylabel(gray_reading(Mask_Picking).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Mask_Picking))

axis[2].set_xlabel(simple_reading(Image_Picking).shape)
axis[2].set_ylabel(simple_reading(Image_Picking).size)
axis[2].set_title("BACTERIA")
axis[2].imshow(simple_reading(Image_Picking))

#### CANNY VISION

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(11,11))


Image_Picking = Main_Set["BACTERIA"][1]
Mask_Picking = Main_Set["MASK"][1]

axis[0].set_xlabel(canny_reading(simple_reading(Image_Picking)).shape)
axis[0].set_ylabel(canny_reading(simple_reading(Image_Picking)).size)
axis[0].set_title("CANNY BASIC")
axis[0].imshow(canny_reading(simple_reading(Image_Picking)))

axis[1].set_xlabel(gray_reading(Mask_Picking).shape)
axis[1].set_ylabel(gray_reading(Mask_Picking).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Mask_Picking))

axis[2].set_xlabel(simple_reading(Image_Picking).shape)
axis[2].set_ylabel(simple_reading(Image_Picking).size)
axis[2].set_title("BACTERIA")
axis[2].imshow(simple_reading(Image_Picking))

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(11,11))


Image_Picking = Main_Set["BACTERIA"][10]
Mask_Picking = Main_Set["MASK"][10]

axis[0].set_xlabel(canny_reading(simple_reading(Image_Picking)).shape)
axis[0].set_ylabel(canny_reading(simple_reading(Image_Picking)).size)
axis[0].set_title("CANNY BASIC")
axis[0].imshow(canny_reading(simple_reading(Image_Picking)))

axis[1].set_xlabel(gray_reading(Mask_Picking).shape)
axis[1].set_ylabel(gray_reading(Mask_Picking).size)
axis[1].set_title("MASK")
axis[1].imshow(gray_reading(Mask_Picking))

axis[2].set_xlabel(simple_reading(Image_Picking).shape)
axis[2].set_ylabel(simple_reading(Image_Picking).size)
axis[2].set_title("BACTERIA")
axis[2].imshow(simple_reading(Image_Picking))

#### HESSIAN VISION

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Max_Image,Min_Image = hessian_reading(Main_Set["BACTERIA"][230],0.15,"rc")

axis[0].set_xlabel(Max_Image.shape)
axis[0].set_ylabel(Max_Image.size)
axis[0].set_title("MAX")
axis[0].imshow(Max_Image)

axis[1].set_xlabel(Min_Image.shape)
axis[1].set_ylabel(Min_Image.size)
axis[1].set_title("MIN")
axis[1].imshow(Min_Image)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Max_Image,Min_Image = hessian_reading(Main_Set["BACTERIA"][7],0.15,"rc")

axis[0].set_xlabel(Max_Image.shape)
axis[0].set_ylabel(Max_Image.size)
axis[0].set_title("MAX")
axis[0].imshow(Max_Image)

axis[1].set_xlabel(Min_Image.shape)
axis[1].set_ylabel(Min_Image.size)
axis[1].set_title("MIN")
axis[1].imshow(Min_Image)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Max_Image,Min_Image = hessian_reading(Main_Set["BACTERIA"][77],0.15,"rc")

axis[0].set_xlabel(Max_Image.shape)
axis[0].set_ylabel(Max_Image.size)
axis[0].set_title("MAX")
axis[0].imshow(Max_Image)

axis[1].set_xlabel(Min_Image.shape)
axis[1].set_ylabel(Min_Image.size)
axis[1].set_title("MIN")
axis[1].imshow(Min_Image)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Max_Image,Min_Image = hessian_reading(Main_Set["BACTERIA"][177],0.15,"rc")

axis[0].set_xlabel(Max_Image.shape)
axis[0].set_ylabel(Max_Image.size)
axis[0].set_title("MAX")
axis[0].imshow(Max_Image)

axis[1].set_xlabel(Min_Image.shape)
axis[1].set_ylabel(Min_Image.size)
axis[1].set_title("MIN")
axis[1].imshow(Min_Image)

#### SKELETON VISION

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Skeleton_Image = skeleton_reading(Main_Set["BACTERIA"][22])

axis[0].set_xlabel(Skeleton_Image.shape)
axis[0].set_ylabel(Skeleton_Image.size)
axis[0].set_title("SKELETON")
axis[0].imshow(Skeleton_Image)

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][22]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][22]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][22]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Skeleton_Image = skeleton_reading(Main_Set["BACTERIA"][229])

axis[0].set_xlabel(Skeleton_Image.shape)
axis[0].set_ylabel(Skeleton_Image.size)
axis[0].set_title("SKELETON")
axis[0].imshow(Skeleton_Image)

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][229]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][229]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][229]))

#### CONTOUR DRAWING VISION

In [ ]:
drawing_contours_vision(Main_Set["BACTERIA"][111])

In [ ]:
drawing_contours_vision(Main_Set["BACTERIA"][161])

In [ ]:
drawing_contours_vision(Main_Set["BACTERIA"][61])

#### SPECTRUM VISION

In [ ]:
simple_spectrum_vision(Main_Set["BACTERIA"][299],"jet")

In [ ]:
simple_spectrum_vision(Main_Set["BACTERIA"][299],"hot")

In [ ]:
simple_spectrum_vision(Main_Set["BACTERIA"][299],"Spectral")

In [ ]:
simple_spectrum_vision(Main_Set["BACTERIA"][99],"jet")

In [ ]:
simple_spectrum_vision(Main_Set["BACTERIA"][99],"hot")

In [ ]:
simple_spectrum_vision(Main_Set["BACTERIA"][99],"Spectral")

#### TRANSFORMATION VISION

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Transformation_Mask = layers_reading(Main_Set["BACTERIA"][22],Main_Set["MASK"][22])

axis[0].set_xlabel(Transformation_Mask.shape)
axis[0].set_ylabel(Transformation_Mask.size)
axis[0].set_title("TRANSFORMATION")
axis[0].imshow(Transformation_Mask)

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][22]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][22]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][22]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Transformation_Mask = layers_reading(Main_Set["BACTERIA"][277],Main_Set["MASK"][277])

axis[0].set_xlabel(Transformation_Mask.shape)
axis[0].set_ylabel(Transformation_Mask.size)
axis[0].set_title("TRANSFORMATION")
axis[0].imshow(Transformation_Mask[:,:,0])

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][277]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][277]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][277]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Transformation_Mask = layers_reading(Main_Set["BACTERIA"][301],Main_Set["MASK"][301])

axis[0].set_xlabel(Transformation_Mask.shape)
axis[0].set_ylabel(Transformation_Mask.size)
axis[0].set_title("TRANSFORMATION")
axis[0].imshow(Transformation_Mask[:,:,0])

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][301]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][301]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][301]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Transformation_Mask = layers_reading(Main_Set["BACTERIA"][201],Main_Set["MASK"][201])

axis[0].set_xlabel(Transformation_Mask.shape)
axis[0].set_ylabel(Transformation_Mask.size)
axis[0].set_title("TRANSFORMATION")
axis[0].imshow(Transformation_Mask[:,:,0])

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][201]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][201]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][201]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Transformation_Mask = layers_reading(Main_Set["BACTERIA"][291],Main_Set["MASK"][291])

axis[0].set_xlabel(Transformation_Mask.shape)
axis[0].set_ylabel(Transformation_Mask.size)
axis[0].set_title("TRANSFORMATION")
axis[0].imshow(Transformation_Mask)

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][291]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][291]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][291]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Transformation_Mask = layers_reading(Main_Set["BACTERIA"][91],Main_Set["MASK"][91])

axis[0].set_xlabel(Transformation_Mask.shape)
axis[0].set_ylabel(Transformation_Mask.size)
axis[0].set_title("TRANSFORMATION")
axis[0].imshow(Transformation_Mask)

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][91]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][91]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][91]))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(11,11))

Transformation_Mask = layers_reading(Main_Set["BACTERIA"][7],Main_Set["MASK"][7])

axis[0].set_xlabel(Transformation_Mask.shape)
axis[0].set_ylabel(Transformation_Mask.size)
axis[0].set_title("TRANSFORMATION")
axis[0].imshow(Transformation_Mask)

axis[1].set_xlabel(simple_reading(Main_Set["BACTERIA"][7]).shape)
axis[1].set_ylabel(simple_reading(Main_Set["BACTERIA"][7]).size)
axis[1].set_title("BACTERIA")
axis[1].imshow(simple_reading(Main_Set["BACTERIA"][7]))

# TRANSFORMATION DATA

#### TRANSFORMATION

In [ ]:
Image_List = []
Transformation_List = []

for image_x,mask_x in zip(Main_Set.BACTERIA.values,Main_Set.MASK.values):
    
    Reading_Image = simple_reading(image_x)
    Transformation_Image = layers_reading(image_x,mask_x)
    
    Resized_Image = cv2.resize(Reading_Image,(180,180))
    Resized_Transformation = cv2.resize(Transformation_Image,(180,180))
    
    Image_List.append(Resized_Image)
    Transformation_List.append(Resized_Transformation)

#### CHECKING

In [ ]:
print(len(Image_List))
print(len(Transformation_List))

In [ ]:
print("WHEN IT IS ARRAY IMAGE SHAPE: ",np.shape(np.array(Image_List)))
print("WHEN IT IS ARRAY MASK SHAPE: ",np.shape(np.array(Transformation_List)))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Transformation_List[10],cmap="jet")
axis[0].set_xlabel(Transformation_List[10].shape)
axis[0].set_title("MASK")
axis[1].imshow(Image_List[10])
axis[1].set_xlabel(Image_List[10].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Transformation_List[100],cmap="jet")
axis[0].set_xlabel(Transformation_List[100].shape)
axis[0].set_title("MASK")
axis[1].imshow(Image_List[100])
axis[1].set_xlabel(Image_List[100].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Transformation_List[190],cmap="jet")
axis[0].set_xlabel(Transformation_List[190].shape)
axis[0].set_title("MASK")
axis[1].imshow(Image_List[190])
axis[1].set_xlabel(Image_List[190].shape)
axis[1].set_title("ORIGINAL")

#### TO ARRAY

In [ ]:
Train_Set = np.array(Image_List,dtype="float32")
Transformation_Set = np.array(Transformation_List,dtype="float32")

Train_Set = Train_Set / 255.
Transformation_Set = Transformation_Set / 255.

In [ ]:
print("TRAIN SHAPE: ",Train_Set.shape)
print("TRANSFORMATION SHAPE: ",Transformation_Set.shape)

In [ ]:
print("TRAIN REDUCE SHAPE: ",Train_Set[:350].shape)
print("TRANSFORMATION REDUCE SHAPE: ",Transformation_Set[:350].shape)

# AUTO ENCODER PROCESS

#### PARAMETERS

In [ ]:
compile_loss = "binary_crossentropy"
compile_optimizer = Adam(lr=0.00001)
output_class = 1

#### CALLBACKS

In [ ]:
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck")

#### STRUCTURE

In [ ]:
E_AE = Sequential()
E_AE.add(Conv2D(20,(5,5),kernel_initializer = 'he_normal',use_bias = True))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())
#
E_AE.add(Conv2D(40,(5,5),kernel_initializer = 'he_normal',use_bias = True))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())
#
E_AE.add(Conv2D(80,(5,5),kernel_initializer = 'he_normal',use_bias = True))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())
#
E_AE.add(Conv2D(160,(5,5),kernel_initializer = 'he_normal',use_bias = True))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())
#
E_AE.add(Conv2D(210,(5,5),kernel_initializer = 'he_normal',use_bias = True))
E_AE.add(BatchNormalization())
E_AE.add(ReLU())


D_AE = Sequential()
D_AE.add(Conv2DTranspose(160,(5,5)))
D_AE.add(ReLU())
#
D_AE.add(Conv2DTranspose(80,(5,5)))
D_AE.add(ReLU())
#
D_AE.add(Conv2DTranspose(40,(5,5)))
D_AE.add(ReLU())
#
D_AE.add(Conv2DTranspose(20,(5,5)))
D_AE.add(ReLU())
#
D_AE.add(Conv2DTranspose(output_class,(5,5)))
D_AE.add(ReLU())

In [ ]:
Auto_Encoder = Sequential([E_AE,D_AE])
Auto_Encoder.compile(loss=compile_loss,optimizer=compile_optimizer,metrics=["mse"])

#### TRAINING

In [ ]:
Model_AutoEncoder = Auto_Encoder.fit(Train_Set[:350],Transformation_Set[:350],epochs=40,callbacks=[Checkpoint_Model])

#### PREDICTION SEEN

In [ ]:
Prediction_Seen = Auto_Encoder.predict(Train_Set[:5])

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Train_Set[4])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Seen[4])
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Train_Set[1])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Seen[1])
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Train_Set[0])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Seen[0])
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Train_Set[2])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Seen[2])
axis[1].set_title("PREDICTION")

#### PREDICTION NON-SEEN / MAIN PREDICTION

In [ ]:
Prediction_Non_Seen = Auto_Encoder.predict(Train_Set[350:])

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Train_Set[350])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen[0])
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Train_Set[351])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen[1])
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Train_Set[355])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen[5])
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Train_Set[360])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen[10])
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(4,4,figsize=(10,10))

for indexing,operations in enumerate(axis.flat):
    
    Picking_Prediction_Image = Prediction_Non_Seen[indexing]
    
    operations.set_xlabel(Picking_Prediction_Image.shape)
    operations.set_ylabel(Picking_Prediction_Image.size)
    operations.set_title("PREDICTION")
    operations.imshow(Picking_Prediction_Image)
    
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(4,4,figsize=(10,10))

for indexing,operations in enumerate(axis.flat):
    
    Picking_Prediction_Image = Prediction_Non_Seen[indexing]
    
    operations.set_xlabel(Picking_Prediction_Image.shape)
    operations.set_ylabel(Picking_Prediction_Image.size)
    operations.set_title("PREDICTION")
    operations.imshow(Picking_Prediction_Image,cmap="hot")
    
plt.tight_layout()
plt.show()